In [ ]:
from transformers import pipeline
from PIL import Image
import requests

# load pipe
image_classifier = pipeline(task="zero-shot-image-classification", model="google/siglip-so400m-patch14-224", device='cuda')

In [10]:
# load image
image0 = Image.open("./Scene/0/0.jpg")
image1 = Image.open("./Scene/0/1.jpg")
images = [image0, image1]

# inference
outputs = image_classifier(images, candidate_labels=['Buildings', 'Forests', 'Glacier', 'Mountains', 'Sea', 'Street'])
for output in outputs:
    print(output[0]['label'])

Street
Buildings


In [ ]:
from datetime import datetime
import pandas as pd
import os

submission = dict({'id_idx': [], 'label': []})
start_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'submission_{start_time}.csv'

for idx in range(810):
    image_path = f"./Scene/0/{idx}.jpg"
    image = Image.open(image_path)
    class_names=['building', 'forest', 'glacier', 'mountain', 'sea', 'street']
    prompts = [f"A photo of a {class_name}" for class_name in class_names]
    
    tokenizer = CLIPTokenizer.from_pretrained(model_name_or_path)
    input_ids = tokenizer(prompts,  return_tensors="pt").input_ids.to('cpu')
    input_pixels = processor(images=image, return_tensors="pt").pixel_values.to('cpu')

    with torch.no_grad(), torch.amp.autocast('cuda'):
        image_features = model.encode_image(input_pixels)
        text_features = model.encode_text(input_ids)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    
    label_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    predict = int(label_probs.argmax())
    
    submission['label'] += [predict]

    if (idx+1) % 10 == 0:
        submission['id_idx'] = list(range(len(submission['label'])))
        pd.DataFrame(submission).to_csv(os.path.join("/home/jinjinjara1022/ajou/submissions/", file_name), index=False)
        print()

submission['id_idx'] = list(range(len(submission['label'])))
pd.DataFrame(submission).to_csv(os.path.join("/home/jinjinjara1022/ajou/submissions/", file_name), index=False)